In [2]:
import numpy as np
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p) 
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad ])
    res = []
    for i in range(0, int((len(x_padded) - len(w_rot))) + 1, s):    
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
print('Conv1d Implementation:', conv1d(x, w, p=2, s=1))
print('NumPy Results:', np.convolve(x, w, mode='same'))

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
NumPy Results: [ 5 14 16 26 24 34 19 22]


In [11]:
import scipy.signal

def conv2d(X, W, p=(0,0), s=(1,1)):
    W_rot = W[::-1,::-1]
    X_padded = np.pad(X, p)
    res = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0])/s[0])+1,s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0],j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

X = np.array([[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]])
W = np.array([[1, 0, 3], [1, 2, 1], [0, 1, 1]])
print(conv2d(X,W,(1,1),(1,1)))

# Built in scipy version
print(scipy.signal.convolve2d(X, W, mode='same'))


[[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]
[[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


In [14]:
import torchvision 
import torch
from torchvision import transforms 
image_path = './'
transform = transforms.Compose([transforms.ToTensor()])

mnist_dataset = torchvision.datasets.MNIST(root=image_path, 
                                           train=True, 
                                           transform=transform, 
                                           download=True)

from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000)) 
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path, 
                                           train=False, 
                                           transform=transform, 
                                           download=False)

                                        

In [15]:
from torch.utils.data import DataLoader


batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle=False)

In [18]:
import torch.nn as nn


model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2))
# model.add_module('relu1', nn.ReLU())  
x = torch.ones((4, 1, 28, 28))
print(model(x))

tensor([[[[ 0.0790, -0.0029, -0.0400,  ..., -0.0400, -0.0235, -0.0342],
          [ 0.1841,  0.0610,  0.1439,  ...,  0.1439,  0.1848,  0.1464],
          [ 0.0509, -0.1605,  0.0255,  ...,  0.0255,  0.2547,  0.1224],
          ...,
          [ 0.0509, -0.1605,  0.0255,  ...,  0.0255,  0.2547,  0.1224],
          [ 0.2347,  0.1442,  0.4046,  ...,  0.4046,  0.5698,  0.3512],
          [-0.0015, -0.1211,  0.1574,  ...,  0.1574,  0.3847,  0.3410]],

         [[ 0.1701,  0.1549,  0.3575,  ...,  0.3575,  0.5777,  0.5475],
          [-0.0703,  0.1043,  0.3627,  ...,  0.3627,  0.7731,  0.9251],
          [-0.0331,  0.2607,  0.5287,  ...,  0.5287,  0.8151,  1.1626],
          ...,
          [-0.0331,  0.2607,  0.5287,  ...,  0.5287,  0.8151,  1.1626],
          [-0.0123,  0.3877,  0.5529,  ...,  0.5529,  0.8174,  1.1072],
          [-0.0592,  0.4253,  0.3943,  ...,  0.3943,  0.5534,  0.8456]],

         [[ 0.2226,  0.3017,  0.2483,  ...,  0.2483,  0.4367,  0.3997],
          [ 0.2729,  0.3427,  